In [38]:
import time
import random

In [39]:
#Input using String where the numbers are seperated by a space ("1 2 3 4 5 6 7 8 0")
def getInput():
    scan = input("Enter the 8-tile puzzle starting configuration:")
    l = list(scan.split(" "))
    return [int(x) for x in l]
#Display
def display(lis):
    print(lis[0]," ",lis[1]," ",lis[2])
    print(lis[3]," ",lis[4]," ",lis[5])
    print(lis[6]," ",lis[7]," ",lis[8])

#function to return empty tile index
def getIndexOfEmpty(puzzle):
    i = 0
    while i < len(puzzle):
        if puzzle[i] == 0:
            return i
        i = i + 1
#Function to get preconditions for actions
def availableActions(puzzle):
    index = getIndexOfEmpty(puzzle)
    actions = [False,False,False,False] #left, right, Up, Down
    if index < 3:
        actions[3] = False
    else:
        actions[3] = True
    if index > 5:
        actions[2] = False
    else:
        actions[2] = True
    if index == 0 or index == 3 or index == 6:
        actions[1] = False
    else:
        actions[1] = True
    if index == 2 or index == 5 or index == 8:
        actions[0] = False
    else:
        actions[0] = True
    return actions
#Swap function action argument can only be 4 variables: Left=0, Right=1, Up=2, Down=3
def swap(actual, action):
    puzzle = actual.copy()
    index = getIndexOfEmpty(puzzle)
    if action == 0:#Left
        puzzle[index] = puzzle[index+1]
        puzzle[index+1] = 0
    elif action == 1:#Right
        puzzle[index] = puzzle[index-1]
        puzzle[index-1] = 0
    elif action == 2:#Up
        puzzle[index] = puzzle[index+3]
        puzzle[index+3] = 0
    elif action == 3:#Down
        puzzle[index] = puzzle[index-3]
        puzzle[index-3] = 0
    return puzzle
#Heuristic Options
#1.Number of Misplaced tiles
def misplacedTiles(puzzle):
    count = 0
    i = 0
    while i < len(puzzle):
        if puzzle[i] != i+1 and puzzle[i] != 0:
            count = count + 1
        i = i + 1
    return count
#2.Manhatten Distance
def mhd(puzzle, index):
    value = puzzle[index]
    if value < 4:
        rowOfValue = 1
    elif value < 7:
        rowOfValue = 2
    else:
        rowOfValue = 3
    if value == 1 or value == 4 or value == 7:
        colOfValue = 1
    elif value == 2 or value == 5 or value == 8:
        colOfValue = 2
    else:
        colOfValue = 3
    current = index + 1
    if current < 4:
        rowOfCurrent = 1
    elif current < 7:
        rowOfCurrent = 2
    else:
        rowOfCurrent = 3
    if current == 1 or current == 4 or current == 7:
        colOfCurrent = 1
    elif current == 2 or current == 5 or current == 8:
        colOfCurrent = 2
    else:
        colOfCurrent = 3
    return abs(rowOfValue-rowOfCurrent)+abs(colOfValue-colOfCurrent)
    
    
def Manhatten_Distance(puzzle):
    count = 0
    i = 0
    while i < len(puzzle):
        if puzzle[i] != i+1 and puzzle[i] != 0:
            count = count + mhd(puzzle,i)
        i = i + 1
    return count


In [40]:
#Creating the tree structure 
#The nodes will be the board at that state and the weight would be the action applied
class StateTree():
    def __init__(self, state, Actions=[]):
        self.state = state
        self.children = list()
        self.actions = Actions
        self.getNChildren = 0
    def addChild(self, state, action):
        listOfActions = self.actions.copy()
        listOfActions.append(action)
        self.children.append(StateTree(state, listOfActions)) 
        self.getNChildren += 1
#function to generate possible children from current node
def addNodes(node):
    #note: actions[0] = Left, actions[1] = right, actions[2] = Up, actions[3] = Down
    actions = availableActions(node.state)
    i = 0
    for a in actions:
        if a:
            newBoard = swap(node.state, i)
            node.addChild(newBoard, i)
        i = i + 1


In [41]:
#function to display result
def displayResult(actions):
    print("Plan is: Start->",end=' ')
    for el in actions:
        if el == 0:
            print("Left->",end=' ')
        elif el == 1:
            print("Right->",end=' ')
        elif el == 2:
            print("Up->",end=' ')
        else:
            print("Down->",end=' ')
    print("Goal State Reached")

In [161]:
#1.Bredth First Search (This search does not use a heuristic)
def BFS(initialState):
    t0 = time.time()
    S1 = StateTree(initial)
    toVisit = [S1]
    UniqueStates = [] 
    i = 0
    while True:
        tree = toVisit[0]
        toVisit.pop(0)
        if goalState == tree.state:
            count = len(UniqueStates)
            t1 = time.time()
            print("Time taken to execute: ", (t1 - t0)*1000,"ms")
            return tree.actions,count
        #add children
        addNodes(tree)
        N = tree.getNChildren
        j = 0 
        while j < N:
            if UniqueStates.count(tree.children[j].state) == 0:
                UniqueStates.append(tree.children[j].state)
                toVisit.append(tree.children[j])
            j += 1
        i += 1
        if time.time() - t0 > 900:
            print("BFS didn't manage to find a solution in 15 mins :(")
            count = len(UniqueStates)
            return [],count

In [125]:
#Function of get the lowest number's index from dict 
def lowest(l):
    i = 1
    index = 0
    while i < len(l):
        if l[index]['priority'] >= l[i]['priority']:
            index = i
        i += 1
    return index
#2.Bredth First Search (This search uses a heuristic)
def GreedyBestFirstSearch(initialState, heuristic):
    #for now let the heuristic be mhd
    t0 = time.time()
    S1 = StateTree(initial)
    find = []
    if heuristic == "mhd":
        find.append({'node':S1,'priority':Manhatten_Distance(S1.state)})
    else:
        find.append({'node':S1,'priority':misplacedTiles(S1.state)})
    UniqueStates = []
    i = 0
    while True:
        index = lowest(find)
        tree = find[index]['node']
        find.pop(index)
        if goalState == tree.state:
            count = len(UniqueStates)
            t1 = time.time()
            print("Time taken to execute: ", (t1 - t0)*1000,"ms")
            return tree.actions,count
        #add children
        addNodes(tree)
        N = tree.getNChildren
        j = 0 
        while j < N:
            if UniqueStates.count(tree.children[j].state) == 0:
                UniqueStates.append(tree.children[j].state)
                if heuristic == "mhd":
                    find.append({'node':tree.children[j],'priority':Manhatten_Distance(tree.children[j].state)})
                else:
                    find.append({'node':tree.children[j],'priority':misplacedTiles(tree.children[j].state)})
            j += 1
        i += 1
        if time.time() - t0 > 900:
            print("Bredth First Search didn't manage to find a solution in 15 mins :(")
            count = len(UniqueStates)
            return [],count


In [92]:
#3.A* (This search uses a heuristic and a cost value)
def Astar(initialState, heuristic):
    t0 = time.time()
    S1 = StateTree(initial)
    find = []
    c = 0
    if heuristic == "mhd":
        h = Manhatten_Distance(S1.state)
        find.append({'node':S1,'priority':c+h})
    else:
        h = misplacedTiles(S1.state)
        find.append({'node':S1,'priority':c+h})
    UniqueStates = []
    i = 0
    while True:
        index = lowest(find)
        tree = find[index]['node']
        find.pop(index)
        if goalState == tree.state:
            count = len(UniqueStates)
            t1 = time.time()
            print("Time taken to execute: ", (t1 - t0)*1000,"ms")
            return tree.actions,count
        #add children
        addNodes(tree)
        N = tree.getNChildren
        j = 0 
        while j < N:
            if UniqueStates.count(tree.children[j].state) == 0:
                UniqueStates.append(tree.children[j].state)
                c = len(tree.children[j].actions)
                if heuristic == "mhd":
                    h = Manhatten_Distance(tree.children[j].state)
                    find.append({'node':tree.children[j],'priority':c+h})
                else:
                    h = misplacedTiles(tree.children[j].state)
                    find.append({'node':tree.children[j],'priority':c+h})
            j += 1
        i += 1
        if time.time() - t0 > 900:
            print("A* didn't manage to find a solution in 15 mins :(")
            count = len(UniqueStates)
            return [],count

In [121]:
#function to check if State is already visited
def NewState(states, node):
    for el in states:
        if el.state == node:
            return False
    return True
#4.Hill Climbing (This search uses a heuristic value and compares it to parent state)
def HillClimbing(initialState, heuristic):
    t0 = time.time()
    S1 = StateTree(initial)
    best = []
    states = []
    visited = []
    if heuristic == "mhd":
        h = Manhatten_Distance(S1.state)
        best.append({'node':S1,'priority':h})
    else:
        h = misplacedTiles(S1.state)
        best.append({'node':S1,'priority':h})
    states.append(S1)
    i = 0
    while True:
        tree = best[0]['node']
        H = best[0]['priority']
        best.pop(0)
        visited.append(tree)
        if goalState == tree.state:
            count = len(states)
            t1 = time.time()
            print("Time taken to execute: ", (t1 - t0)*1000,"ms")
            return tree.actions,count
        #add children
        addNodes(tree)
        N = tree.getNChildren
        addToOpenList = []
        j = 0
        while j < N:
            if NewState(visited,tree.children[j].state):
                states.append(tree.children[j])
                if heuristic == "mhd":
                    h = Manhatten_Distance(tree.children[j].state)
                else:
                    h = misplacedTiles(tree.children[j].state)
                if h < H:
                    best = []
                    best.append({'node':tree.children[j],'priority':h})
                    H = h
                else:
                    addToOpenList.append({'node':tree.children[j],'priority':h})
            j += 1
        #Case where no better heuristic is found
        if len(best) != 1:
            best = best + addToOpenList 
        i += 1
        if time.time() - t0 > 900:
            print("Hill Climbing didn't manage to find a solution in 15 mins :(")
            count = len(states)
            return [],count


In [118]:
#function to validate plan
def isValid(s1,plan):
    i = 0
    state = s1
    print("----------")
    display(s1)
    print("----------")
    while i < len(plan):
        state = swap(state, plan[i])
        display(state)
        print("----------")
        i += 1
    if state == goalState:
        return True
    return False

In [162]:
#"Main" Function
initial = getInput()
goalState = [1,2,3,4,5,6,7,8,0]
display(initial)
print("Choose one of the following search algorithms:\n1.Breadth First Search\n2.Greedy Best First Search\n3.A* Search\n4.Enforced Hill Climbing")
search = int(input(''))
if search != 1:
    print("Using as heuristic: 1.Number of Misplaced Tiles\n2.Manhatten Distance")
    func = int(input(''))
if search == 1:
    print("****Breadth First Search****")
    plan,count = BFS(initial)
    print("The length of the plan is ",len(plan))
    print("The number of Unique States Generated",count)
    displayResult(plan)
    if(isValid(initial,plan)):
          print("VALID Plan")
    else:
          print("Not Valid Plan")
elif search == 2:
    print("****Greedy Best First Search****")
    if func == 1:
        plan, count = GreedyBestFirstSearch(initial,"misplaced")
    else:
        plan, count = GreedyBestFirstSearch(initial,"mhd")
    print("The length of the plan is ",len(plan))
    print("The number of Unique States Generated",count)
    displayResult(plan)
    if(isValid(initial,plan)):
          print("VALID Plan")
    else:
          print("Not Valid Plan")
elif search == 3:
    print("**** A* Search****")
    if func == 1:
        plan, count = Astar(initial,"misplaced")
    else:
        plan, count = Astar(initial,"mhd")
    print("The length of the plan is ",len(plan))
    print("The number of Unique States Generated",count)
    displayResult(plan)
    if(isValid(initial,plan)):
          print("VALID Plan")
    else:
          print("Not Valid Plan")
elif search == 4:
    print("****Enforced Hill Climbing Search****")
    if func == 1:
        plan, count = HillClimbing(initial,"misplaced")
    else:
        plan, count = HillClimbing(initial,"mhd")
    print("The length of the plan is ",len(plan))
    print("The number of Unique States Generated",count)
    displayResult(plan)
    if(isValid(initial,plan)):
          print("VALID Plan")
    else:
          print("Not Valid Plan")
else:
    print("invalid option given")

Enter the 8-tile puzzle starting configuration:1 2 0 4 5 3 7 8 6
1   2   0
4   5   3
7   8   6
Choose one of the following search algorithms:
1.Breadth First Search
2.Greedy Best First Search
3.A* Search
4.Enforced Hill Climbing
1
****Breadth First Search****
Time taken to execute:  0.0 ms
The length of the plan is  2
The number of Unique States Generated 14
Plan is: Start-> Up-> Up-> Goal State Reached
----------
1   2   0
4   5   3
7   8   6
----------
1   2   3
4   5   0
7   8   6
----------
1   2   3
4   5   6
7   8   0
----------
VALID Plan
